In [1]:
"""
Example script that scrapes data from the IEM ASOS download service
"""
from __future__ import print_function
import json
import time
from datetime import datetime, timedelta
#import datetime
import pandas as pd

# Python 2 and 3: alternative 4
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen

# Number of attempts to download data
MAX_ATTEMPTS = 6
# HTTPS here can be problematic for installs that don't have Lets Encrypt CA
SERVICE = "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"


def download_data(uri):
    """Fetch the data from the IEM
    The IEM download service has some protections in place to keep the number
    of inbound requests in check.  This function implements an exponential
    backoff to keep individual downloads from erroring.
    Args:
      uri (string): URL to fetch
    Returns:
      string data
    """
    attempt = 0
    while attempt < MAX_ATTEMPTS:
        try:
            data = urlopen(uri, timeout=300).read().decode("utf-8")
            if data is not None and not data.startswith("ERROR"):
                return data
        except Exception as exp:
            print("download_data(%s) failed with %s" % (uri, exp))
            time.sleep(5)
        attempt += 1

    print("Exhausted attempts to download, returning empty data")
    return ""


def get_stations_from_filelist():
    """Build a listing of stations from a simple file listing the stations.
    The file should simply have one station per line.
    """
    stations = []
    for line in open(filename):
        stations.append(line.strip())
    return stations


def get_stations_from_networks():
    """Build a station list by using a bunch of IEM networks."""
    stations = ["ATL"]
    states = "GA"
    # IEM quirk to have Iowa AWOS sites in its own labeled network
    networks = ["AWOS"]
    for state in states.split():
        networks.append("%s_ASOS" % (state,))

    for network in networks:
        # Get metadata
        uri = (
            "https://mesonet.agron.iastate.edu/" "geojson/network/%s.geojson"
        ) % (network,)
        data = urlopen(uri)
        jdict = json.load(data)
        for site in jdict["features"]:
            stations.append(site["properties"]["sid"])
    return stations


def main():
    
    """Our main method"""
    # timestamps in UTC to request data for
    startts = datetime.utcnow().date() - timedelta(days=1095)#datetime.datetime(2020, 6, 15)
    endts   = datetime.utcnow().date()
    
    
    # Two examples of how to specify a list of stations
    #stations = get_stations_from_networks()
    # stations = get_stations_from_filelist("mystations.txt")
    
    stations = ["ATL"]
    dfs = []   
    data_types = ["mslp", "tmpf"]
    for station in stations:
        
        for data_type in data_types:
     
            service = SERVICE + "data="+data_type+"&tz=Etc/UTC&format=onlytdf&=yes&"

            service += startts.strftime("year1=%Y&month1=%m&day1=%d&")
            service += endts.strftime("year2=%Y&month2=%m&day2=%d&")
    

            
            uri = "%s&station=%s" % (service, station)
            print("Downloading: %s" % (station,))
            data = download_data(uri)
            outfn = "%s_%s_%s.txt" % (
                station,
                startts.strftime("%Y%m%d%H%M"),
                endts.strftime("%Y%m%d%H%M"),
            )
            out = open(outfn, "w")
            out.write(data)
            out.close()
            
            df = pd.read_csv(outfn, sep= '\t' )


            out.close()
            dfs.append(df)




            df.head()
            print(df)
            


    
    final_dataframe = pd.merge(dfs[0], dfs[1])
    print(final_dataframe)
    
    final_dataframe.to_csv("mslp_temp.csv",index = False)




if __name__ == "__main__":
    main()

Downloading: ATL
       station             valid     mslp
0          ATL  2021-06-03 00:00        M
1          ATL  2021-06-03 00:05        M
2          ATL  2021-06-03 00:10        M
3          ATL  2021-06-03 00:15        M
4          ATL  2021-06-03 00:20        M
...        ...               ...      ...
342308     ATL  2024-06-01 23:35        M
342309     ATL  2024-06-01 23:40        M
342310     ATL  2024-06-01 23:45        M
342311     ATL  2024-06-01 23:50        M
342312     ATL  2024-06-01 23:52  1018.60

[342313 rows x 3 columns]
Downloading: ATL
       station             valid   tmpf
0          ATL  2021-06-03 00:00      M
1          ATL  2021-06-03 00:05      M
2          ATL  2021-06-03 00:10      M
3          ATL  2021-06-03 00:15      M
4          ATL  2021-06-03 00:20      M
...        ...               ...    ...
342308     ATL  2024-06-01 23:35      M
342309     ATL  2024-06-01 23:40      M
342310     ATL  2024-06-01 23:45      M
342311     ATL  2024-06-01 23:50   